# EV Charging Stations

In [283]:
import requests
import pandas as pd
import folium

In [284]:
def get_charging_stations(latitude, longitude, radius, api_key):
    url = f"https://api.openchargemap.io/v3/poi/?output=json&latitude={latitude}&longitude={longitude}&distance={radius}&distanceunit=KM&maxresults=300&&verbose=false&key={api_key}"
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        return data

    except requests.exceptions.RequestException as e:
        print("Error occurred while retrieving charging station data:", e)
        return None
    except ValueError as e:
        print("Error occurred while parsing JSON response:", e)
        return None

# Dublin Center coordinates
latitude = 53.349805
longitude = -6.26031

# Radius in kilometers
radius = 25

# API key from OpenChargeMap
api_key = "431e2548-cd50-4959-971b-5e99539c2651"

charging_stations = get_charging_stations(latitude, longitude, radius, api_key)

if charging_stations:
    # Convert charging_stations to DataFrame
    df_charging_stations = pd.DataFrame(charging_stations)

    # Save DataFrame to CSV
    df_charging_stations.to_csv('charging_stations.csv', index=False)

In [285]:
len(df_charging_stations)

243

In [286]:
df_1 = df_charging_stations.loc[:, ['UUID', 'AddressInfo']]
df_2 = df_charging_stations.loc[:, ['UUID', 'Connections']]

In [287]:
# Removing the record without any connection info
df_2 = df_2[df_2['Connections'].apply(lambda x: len(x) != 0)]
df_1 = df_1[df_1['UUID'] != '56834D2E-FBF9-4CDF-BCCE-A0015C49ED3C']

In [288]:
arr1 = []

for index, row in df_1.iterrows():
    # Access values of each column for the current row
    val1 = row['UUID']
    val2 = [row['AddressInfo']]
    
    for dictionary in val2:
        dictionary['UUID'] = val1
        arr1.append(dictionary)

arr1 = pd.DataFrame(arr1)

In [289]:
arr2 = []

for index, row in df_2.iterrows():
    # Access values of each column for the current row
    val1 = row['UUID']
    val2 = row['Connections']
    val3 = val2[0]['ConnectionType']
    
    for dictionary in val2:
        dictionary['UUID'] = val1
        dictionary['Name'] = val3['Title']
        arr2.append(dictionary)
        
arr2 = pd.DataFrame(arr2)

In [290]:
merged_df = arr1.merge(arr2, on='UUID', how='left')

In [291]:
len(merged_df)

331

In [292]:
merged_df.to_csv('final_charging_stations.csv', index=False)

In [293]:
merged_df['Name'].unique()

array(['Type 2 (Socket Only)', 'Tesla (Model S/X)', 'CCS (Type 2)',
       'CHAdeMO', 'Type 1 (J1772)', 'BS1363 3 Pin 13 Amp',
       'Tesla Supercharger'], dtype=object)

In [294]:
# Define a dictionary to map colors to categories
color_map = {
    'Type 1 (J1772)': 'orange',
    'Type 2 (Socket Only)': 'blue',
    'CCS (Type 2)': 'red',
    'Tesla (Model S/X)': 'green',
    'CHAdeMO': 'gray',
    'BS1363 3 Pin 13 Amp': 'purple',
    'Tesla Supercharger': 'darkgreen'
}

# Create a map centered around Dublin to display the EV charging points
map_osm = folium.Map(location=[latitude, longitude], zoom_start=12)

# Add markers for each charging station location
for index, row in merged_df.iterrows():
    name = row['Name'] + ', ' + row['Title'] + ', ' + row['AddressLine1'] + ', ' + str(row['Town'])
    lon = row['Longitude']
    lat = row['Latitude']
    
    category = row['Name']
    marker_color = color_map.get(category, 'gray')  # Use 'gray' if category not found in the color map

    # marker = folium.Marker(location=[lat, lon], popup=name)
    # marker = folium.CircleMarker(location=[lat, lon], radius=5, color=marker_color, fill=True, fill_color=marker_color, fill_opacity=1, popup=name)
    icon = folium.Icon(color=marker_color, icon='map-marker')  # Use the 'map-marker' icon with the specified color

    marker = folium.Marker(location=[lat, lon], icon=icon, popup=name)

    marker.add_to(map_osm)

# Display the map
map_osm